In [1]:
import os
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from Atools import functions
from Atools.base import Experiment

# Plots and load
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)
plt.rc('font', size=10)
plt.rc('axes', titlesize=16)
plt.rc('axes', labelsize=15)
plt.rc('legend', fontsize=10)
plt.rcParams["figure.figsize"] = (10, 7)

In [ ]:
VERBOSE_COLORS = {
    'OK': 'g',
    'OK_inner': 'b',
    'image_sweep_check': 'r',
    'combine_check': 'w',
}


def temperature_plot(
    experiment: base.Experiment,
    title='',
    xlabel='',
    ylabel='',
    interactive=False,
    save_folder=None,
):
    fig, ax = plt.subplots()
    colors = experiment.d['Viscosity_verbose'].replace(VERBOSE_COLORS)
    ax.scatter(x=experiment.d['x'], y=experiment.d['y'], c=colors, s=5)
    sns.lineplot(
        ax=ax,
        data=experiment.d,
        x='x',
        y='y',
        estimator='mean',
        errorbar=("sd", 1),
        label='mean',
    )
    sns.lineplot(
        ax=ax,
        data=experiment.d,
        x="x",
        y="y",
        errorbar=('pi', 50),
        estimator="median",
        label='median',
    )

    fig.canvas.manager.set_window_title(title + ' plot')
    fig.subplots_adjust(
        top=0.9,
        bottom=0.1,
        left=0.1,
        right=0.9,
        hspace=0.2,
        wspace=0.2,
    )
    ax.set_title(f"{experiment.name}: ({experiment.info['w']}% mass)")
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)

    if interactive:
        plt.show()
    if save_folder is not None:
        os.makedirs(f'{save_folder}\Plots', exist_ok=True)
        fig.savefig(f'{save_folder}\Plots\\{title}_{experiment.name}.jpg', dpi=600)


def comparation_plot(
    experiment: base.Experiment,
    ols_exp: base.Experiment,
    title,
    xlabel,
    ylabel,
    interactive=False,
    save_folder=None,
):
    fig, ax = plt.subplots()
    colors = experiment.d['Viscosity_verbose'].replace(VERBOSE_COLORS)
    ax.scatter(
        experiment.d['x'],
        experiment.d['y'],
        color=colors,
        marker='.',
        label='Data',
        alpha=0.6,
    )

    E = experiment.info['E']
    D0 = experiment.info['D0']
    ax.plot(
        ols_exp.d['x'],
        ols_exp.d['y'],
        color='black',
        label=f'OLS: \nE= {E/1000: >8.2f} kJ\nD= {D0: >8.2e} m2/s',
    )

    fig.canvas.manager.set_window_title(title + ' plot')
    fig.subplots_adjust(
        top=0.9,
        bottom=0.1,
        left=0.1,
        right=0.9,
        hspace=0.2,
        wspace=0.2,
    )

    ax.set_title(f"{experiment.name}: ({experiment.info['w']}% mass)")
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.legend()

    if interactive:
        plt.show()
    if save_folder is not None:
        os.makedirs(f'{save_folder}\Plots', exist_ok=True)
        fig.savefig(f'{save_folder}\Plots\\{title}_{experiment.name}.jpg', dpi=600)

In [ ]:
def regress(experiment: Experiment):
    experiment = experiment.apply(functions.C_to_K).apply(functions.linearize)

    df = experiment.d
    df["x0"] = 1
    result = OLS(df["y"], df[["x", "x0"]]).fit()
    means = result.params

    D0 = np.exp(means["x0"])
    E = -8.314 * means["x"]

    conf_int = result.conf_int(0.005).loc
    conf_int["x0"] = np.exp(conf_int["x0"])
    dD0 = (conf_int["x0"].max() - conf_int["x0"].min()) / 2
    conf_int["x"] = -8.314 * conf_int["x"]
    dE = (conf_int["x"].max() - conf_int["x"].min()) / 2

    info = dict(
        E=E,
        D0=D0,
        dD0=dD0,
        dE=dE,
        f_statistic=result.fvalue,
        r2=result.rsquared,
    )
    func = lambda T: D0 * np.exp(-E / (8.314 * T))
    return info, result, func


def create_OLS(exp: Experiment):
    info, result, func = regress(exp)
    exp.set_info(**info)
    x = np.linspace(13, 42, 100) + 273.15
    ols_res = Experiment(
        pd.DataFrame(
            {
                "x": x,
                "y": func(x),
                "time": x * 0,
            }
        ),
        "interpolated",
    )
    return ols_res.apply(functions.K_to_C)


def initial_filter(df, x=(-np.inf, np.inf), y=(0, np.inf), time=(0, np.inf)):
    temperature_cond = (x[0] < df["x"]) & (df["x"] < x[1])
    viscosity_cond = (y[0] < df["y"]) & (df["y"] < y[1])
    time_cond = (time[0] < df["time"]) & (df["time"] < time[1])
    return df[temperature_cond & viscosity_cond & time_cond]


def ask_continue():
    res = None
    while res is None:
        ask = input("Continue [y] and n: ")
        if ask in ["", "y"]:
            res = True
        elif ask in ["n"]:
            res = False
        else:
            print("Incorrect input!")
    return res


def temporal_plot(
    experiment: base.Experiment,
    title="",
    ylabel="",
    interactive=False,
    save_folder=None,
):
    fig, ax_v = plt.subplots()
    ax_T = ax_v.twinx()
    ax_v.scatter(experiment.d["time"], experiment.d["y"], color="red", marker=".")
    ax_T.scatter(experiment.d["time"], experiment.d["x"], color="blue", marker=".")

    fig.canvas.manager.set_window_title(title + " plot")
    fig.subplots_adjust(
        top=0.9,
        bottom=0.1,
        left=0.1,
        right=0.9,
        hspace=0.2,
        wspace=0.2,
    )
    ax_T.set_title(f"{experiment.name}: ({experiment.info['w']}% mass)")
    ax_v.set_xlabel("Time [s]")
    ax_T.set_ylabel("Temperature [C]", color="blue")
    ax_v.set_ylabel(ylabel, color="red")

    if interactive:
        plt.show()
    if save_folder is not None:
        os.makedirs(f"{save_folder}\Plots", exist_ok=True)
        fig.savefig(f"{save_folder}\Plots\\{title}_{experiment.name}.jpg", dpi=600)


def configurate_data(experiment: Experiment) -> Experiment:
    while True:
        exp = experiment.copy()
        time_lim = ()
        while len(time_lim) != 2:
            time_lim = input("Time lim (space as delimiter): ")
            time_lim = [float(i) for i in time_lim.split(" ") if "" != i]
            if len(time_lim) == 1:
                time_lim.append(np.inf)

        y_lim = ()
        while len(y_lim) != 2:
            y_lim = input("Viscosity lim (space as delimiter): ")
            y_lim = [float(i) for i in y_lim.split(" ")]

        exp.d = initial_filter(exp.d, time=time_lim, y=y_lim, x=(12, 42))
        exp.log.append(("initial_filter", {"time": time_lim, "y": y_lim, "x": (12, 42)}))

        temporal_plot(
            exp,
            title="Configurate",
            ylabel="Viscosity [cP]",
            interactive=True,
        )
        if ask_continue():
            break
    return exp

In [3]:
viscosity_path=Path('Viscosity')

In [ ]:
folders = [viscosity_path/i
    for i in os.listdir(viscosity_path)
    if os.path.isdir(viscosity_path/i) ]

for folder in folders:
    exp = Experiment()

    file = [file for file in os.listdir(folder) if ".hdf5" in file][0]

    exp.load_hdf5(input_path(f"{folder}\{file}"))
    exp.apply(functions.C_to_K)
    exp.apply(functions.nu_D)
    exp.apply(functions.K_to_C)
    # exp.folder='.'

    temperature_plot(
        exp,
        title="Temperature",
        xlabel="Temperature [C]",
        ylabel="Viscosity [cP]",
        # ylabel='Diffusion [m2/s]',
        interactive=True,
        save_folder=exp.folder,
    )

In [ ]:
exp = Experiment()
exp.load_hdf5(input_path())
# exp.apply(funcs.C_to_K)
# exp.apply(funcs.nu_D)
# exp.apply(funcs.K_to_C)

info, result, func = tools.regress(exp)
exp.set_info(**info)
x = np.linspace(13, 42, 100) + 273.15
ols_res = tools.Experiment(
    pd.DataFrame({"x": x, "y": func(x), "time": x * 0}),
    "interpolated",
)

ols_res.apply(funcs.K_to_C)

plots.comparation_plot(
    exp,
    ols_res,
    title="OLS_Diffusion",
    xlabel="Temperature [C]",
    ylabel="D [m2/s]",
    interactive=True,
    save_folder=exp.folder,
)
# plots.temperature_plot(
#     exp,
#     title='Viscosity',
#     xlabel='Temperature [C]',
#     ylabel='Viscosity [cP]',
#     # ylabel='Diffusion [m2/s]',
#     interactive=True,
#     save_folder=exp.folder,
# )